In [1]:
import os
import re
import pandas as pd

# === CONFIG ===
BASE_RUN_DIR = r"F:\Code\random-plotter\hasil\StationaryClustered\run"
OUTPUT_BASE_DIR = os.path.join("data", "stationaryClustered", "csv")

# Keys to extract from filename
filename_keys = ["Agent_movementModel", "Target_movementModel", "LevyFlight.alpha"]

# === HELPERS ===
def parse_filename_metadata(filename, keys_to_extract):
    base = os.path.splitext(filename)[0]
    metadata = {'filename': base}
    for key in keys_to_extract:
        pattern = rf'{re.escape(key)}@([^\-_@]+)'
        match = re.search(pattern, base)
        metadata[key] = match.group(1) if match else ''
    return metadata

def parse_csv_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if df.empty or "percentage" not in df.columns:
            return None

        result = {
            "min_coverage": df["percentage"].min(),
            "avg_coverage": df["percentage"].mean(),
            "max_coverage": df["percentage"].max()
        }

        for target in [25, 50, 75, 90]:
            time_reached = df[df["percentage"] >= target]
            result[f"time_to_{target}_percent"] = (
                time_reached["time"].iloc[0] if not time_reached.empty else 0
            )

        return result
    except Exception as e:
        print(f"❌ Failed to process {filepath}: {e}")
        return None

def extract_simulation_summaries_for_folder(folder_path, folder_number):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv') and "SearchingCoverageOverTimeReport" in filename:
            filepath = os.path.join(folder_path, filename)
            metadata = parse_filename_metadata(filename, filename_keys)
            stats = parse_csv_file(filepath)
            if stats is None:
                stats = {
                    "min_coverage": 0,
                    "avg_coverage": 0,
                    "max_coverage": 0,
                    "time_to_25_percent": 0,
                    "time_to_50_percent": 0,
                    "time_to_75_percent": 0,
                    "time_to_90_percent": 0
                }
            all_data.append({**metadata, **stats})

    df = pd.DataFrame(all_data)

    # Convert alpha to float if possible
    if "LevyFlight.alpha" in df.columns:
        df["LevyFlight.alpha"] = pd.to_numeric(df["LevyFlight.alpha"], errors='coerce')

    # Sort for consistency
    df = df.sort_values(by=["Agent_movementModel", "Target_movementModel", "LevyFlight.alpha"])

    # Save CSV
    os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
    output_csv = os.path.join(OUTPUT_BASE_DIR, f"detailed_summary_report_{folder_number}.csv")
    df.to_csv(output_csv, index=False)
    print(f"✅ Extracted summary for folder {folder_number} -> {output_csv}")

# === MAIN LOOP ===
def main():
    for folder_name in os.listdir(BASE_RUN_DIR):
        folder_path = os.path.join(BASE_RUN_DIR, folder_name)
        if os.path.isdir(folder_path) and folder_name.isdigit():
            extract_simulation_summaries_for_folder(folder_path, folder_name)

if __name__ == "__main__":
    main()


✅ Extracted summary for folder 1 -> data\stationaryClustered\csv\detailed_summary_report_1.csv
✅ Extracted summary for folder 10 -> data\stationaryClustered\csv\detailed_summary_report_10.csv
✅ Extracted summary for folder 11 -> data\stationaryClustered\csv\detailed_summary_report_11.csv
✅ Extracted summary for folder 12 -> data\stationaryClustered\csv\detailed_summary_report_12.csv
✅ Extracted summary for folder 13 -> data\stationaryClustered\csv\detailed_summary_report_13.csv
✅ Extracted summary for folder 14 -> data\stationaryClustered\csv\detailed_summary_report_14.csv
✅ Extracted summary for folder 15 -> data\stationaryClustered\csv\detailed_summary_report_15.csv
✅ Extracted summary for folder 16 -> data\stationaryClustered\csv\detailed_summary_report_16.csv
✅ Extracted summary for folder 17 -> data\stationaryClustered\csv\detailed_summary_report_17.csv
✅ Extracted summary for folder 18 -> data\stationaryClustered\csv\detailed_summary_report_18.csv
✅ Extracted summary for folder 1